# Seasonal Decompose Feature Extractor

* Seasonal Strength, Trend Strength, Residual Variability

* Source Link via Rob Hyndman: 
    * https://otexts.com/fpp2/seasonal-strength.html

In [9]:
import hvplot.polars  # noqa
import polars as pl

from polars_ts.decomposition.seasonal_decompose_features import seasonal_decompose_features

In [10]:
df = (
    pl.scan_parquet("https://datasets-nixtla.s3.amazonaws.com/m5_y.parquet")
    .filter(pl.col("unique_id").is_in(pl.col("unique_id").unique().sample(5)))
    .collect()
)

/var/folders/13/qc_45f5n1_v9l3cstfcdl36w0000gn/T/ipykernel_81894/1607245340.py:4: CategoricalRemappingWarning: Local categoricals have different encodings, expensive re-encoding is done to perform this merge operation. Consider using a StringCache or an Enum type if the categories are known in advance
  .collect()


# Plot Results

* Now we see a relative measure of Seasonality/Trend Over All Series.

In [11]:
simple_decompose_features = seasonal_decompose_features(df=df, ts_freq=365, mode="simple")
mstl_decompose_features = seasonal_decompose_features(df=df, ts_freq=365, mode="mstl", seasonal_freqs=[7, 28, 90])

# Plot Naive Decomposition Features

* 24 period seasonality 
* Simple Moving Average Trend (Naive)

In [12]:
seas = simple_decompose_features.hvplot.hist(
    y=["seasonal_strength", "trend_strength"], alpha=0.6, bins=50, title="Hyndman's Seasonal and Trend Strength"
)

seas

:NdOverlay   [Variable]
   :Histogram   [trend_strength]   (Count)

# Plot MSTL Features 
- Multiple Seasonal Components can be examined 
- LOESS Trend as Opposed to MA Trend

In [13]:
# Assuming `mstl_decompose_feats` is a DataFrame containing the columns
# 'seasonal24_strength', 'seasonal48_strength', 'seasonal167_strength'

# Customizing the plot
hist_plot = mstl_decompose_features.hvplot.hist(
    y=["seasonal7_strength", "seasonal28_strength", "seasonal90_strength"],
    title="Normalized MSTL Components of All Series",
    alpha=0.6,  # Transparency for the bars
    bins=50,  # Number of bins
    xlabel="Seasonal Strength S(t)",  # Label for the x-axis
    ylabel="Frequency",  # Label for the y-axis
    grid=True,  # Enable gridlines
)

# Plot trend strength
trend_plot = mstl_decompose_features.hvplot.hist(
    y=["trend_strength"],
    title="Normalized LOESS Component Of All Series",
    alpha=0.6,  # Transparency for the bars
    bins=50,  # Number of bins
    xlabel="Trend Strength T(t)",  # Label for the x-axis
    ylabel="Frequency",  # Label for the y-axis
    grid=True,  # Enable gridlines
)

# Display the plot
(hist_plot + trend_plot).cols(1)

:Layout
   .NdOverlay.I  :NdOverlay   [Variable]
      :Histogram   [seasonal90_strength]   (Count)
   .NdOverlay.II :NdOverlay   [Variable]
      :Histogram   [trend_strength]   (Count)